In [27]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.animation as animation
import matplotlib
from IPython.display import HTML
import plotly.graph_objects as go  #plotly 4.0.0rc1
from plotly.subplots import make_subplots
from matplotlib import pyplot
import plotly.figure_factory as ff
import os

In [28]:
import itertools
import wandb
print(wandb.__path__)
api = wandb.Api()

from collections import OrderedDict

def get_config(runs):
    config_list = [{k:v for k,v in run.config.items() if not k.startswith('_')} for run in runs]
    return pd.DataFrame.from_records(config_list)

RELEVANT_CFG_KEYS = {
'alpha',
}

def get_history_for_key(runs, key, nan_filler='avg_regret'):
    runs = list(runs)
    history = []
    print(len(runs))
    for count, run in enumerate(runs):
        if count % max(1,int(len(runs)/100)) == 0:
            print(count/len(runs)*100, "%")
        history.append(pd.DataFrame.from_records(run.scan_history(keys=[key])).dropna().reset_index(drop=True))
    config_list = [OrderedDict([(k,str(v)) for k,v in run.config.items() if k in RELEVANT_CFG_KEYS])
                    for run in runs]
    keys = [tuple([run.name] + list(config.values())) for run, config in zip(runs, config_list)]
    names = ['Run'] + list(config_list[0].keys()) + ['LogValue']
    df = pd.concat(history, axis=1, names=names, keys=keys)
    nan_filler = {key: run.summary[nan_filler] for run, key in zip(runs, keys)}
    df = df.fillna(nan_filler)
    return df

['/home/francesco/.local/lib/python3.7/site-packages/wandb']


In [29]:
filter = {
    "sweep": {"$in": ['xy64mjxk', 'st3m67nv']},
#    "config.num_updates": 5,
#    "config.learning_rate": 0.5,
#    "config.noise_policy": {"$in": [1.0, 0.1]},
#    "config.run": 2,
}

runs = api.runs('idsia/renyibounds-multi_armed_bandit', filter)
df = get_history_for_key(runs, 'avg_regret')


120
0.0 %
0.8333333333333334 %
1.6666666666666667 %
2.5 %
3.3333333333333335 %
4.166666666666666 %
5.0 %
5.833333333333333 %
6.666666666666667 %
7.5 %
8.333333333333332 %
9.166666666666666 %
10.0 %
10.833333333333334 %
11.666666666666666 %
12.5 %
13.333333333333334 %
14.166666666666666 %
15.0 %
15.833333333333332 %
16.666666666666664 %
17.5 %
18.333333333333332 %
19.166666666666668 %
20.0 %
20.833333333333336 %
21.666666666666668 %
22.5 %
23.333333333333332 %
24.166666666666668 %
25.0 %
25.833333333333336 %
26.666666666666668 %
27.500000000000004 %
28.333333333333332 %
29.166666666666668 %
30.0 %
30.833333333333336 %
31.666666666666664 %
32.5 %
33.33333333333333 %
34.166666666666664 %
35.0 %
35.833333333333336 %
36.666666666666664 %
37.5 %
38.333333333333336 %
39.166666666666664 %
40.0 %
40.833333333333336 %
41.66666666666667 %
42.5 %
43.333333333333336 %
44.166666666666664 %
45.0 %
45.83333333333333 %
46.666666666666664 %
47.5 %
48.333333333333336 %
49.166666666666664 %
50.0 %
50.8333

/home/francesco/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
print(df)

Run      royal-sweep-90 splendid-sweep-89 misunderstood-sweep-88  \
alpha        1000000000        1000000000             1000000000   
LogValue     avg_regret        avg_regret             avg_regret   
0              1.745621          1.298417               1.315765   
1              1.745621          1.298417               1.315765   
2              1.125186          2.366123               1.694407   
3              1.210420          3.639521               2.018046   
4              1.559723          3.310333               2.007384   
..                  ...               ...                    ...   
995            0.944011         -0.463270               0.967081   
996            0.170224          0.974304               0.281468   
997            0.918746          0.369062               0.373483   
998            0.415632          0.827919               0.851376   
999            0.250699          1.167631               0.343946   

Run      good-sweep-87 fluent-sweep-86 serene-s

In [31]:
df = df[1:1001]
df = df.transpose()
df

,,,1,2,3,4,5,6,7,8,9,10,...,990,991,992,993,994,995,996,997,998,999
Run,alpha,LogValue,,,,,,,,,,,,,,,,,,,,,
royal-sweep-90,1000000000,avg_regret,1.745621,1.125186,1.210420,1.559723,0.906299,-0.241534,0.695394,0.111574,0.547342,0.631424,...,0.325147,1.350875,2.094278,0.511252,0.186236,0.944011,0.170224,0.918746,0.415632,0.250699
splendid-sweep-89,1000000000,avg_regret,1.298417,2.366123,3.639521,3.310333,3.381919,3.033132,2.601956,3.727686,3.113230,3.338893,...,0.656451,0.141557,0.240138,1.364905,0.731062,-0.463270,0.974304,0.369062,0.827919,1.167631
misunderstood-sweep-88,1000000000,avg_regret,1.315765,1.694407,2.018046,2.007384,1.066046,1.266929,1.702816,1.374489,1.429591,1.514021,...,0.710150,0.146779,0.204033,0.681265,1.068955,0.967081,0.281468,0.373483,0.851376,0.343946
good-sweep-87,1000000000,avg_regret,1.329634,1.990796,0.702264,-0.472851,1.222924,0.562505,-0.188336,0.795613,-0.474008,0.172244,...,0.461802,-0.104981,0.337336,0.951761,0.524134,0.237414,0.376589,0.230183,0.691893,0.237068
fluent-sweep-86,1000000000,avg_regret,0.748663,3.398654,1.854353,2.424316,2.657235,2.738677,2.707588,1.857832,2.238281,1.609309,...,-0.101996,0.323967,0.585767,0.089513,0.598972,-0.626541,0.192605,-0.066184,-0.459492,-0.056008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dainty-sweep-6,0.5,avg_regret,1.819759,1.957609,0.413191,1.847348,1.377739,1.239103,1.252984,0.342974,2.949355,1.254563,...,1.164221,0.925905,0.548381,0.071442,0.870727,-0.332393,0.432443,0.020089,-1.208340,1.230500
pleasant-sweep-4,1e-05,avg_regret,1.752753,0.709971,2.756819,2.283046,1.624456,1.547539,1.917220,1.195258,1.455468,0.859347,...,1.767929,0.373293,1.383132,0.763825,0.929130,0.300681,0.325776,0.721343,1.622025,0.478680
polar-sweep-3,1e-05,avg_regret,0.562580,1.222737,1.829531,1.855377,2.566259,0.341895,1.783720,1.938878,0.551160,1.654594,...,0.512391,1.849802,1.764964,1.034328,1.294871,0.402750,0.490803,0.807801,1.203387,1.219515


In [32]:
df.to_hdf('data_mab.h5', 'df')